# READ ME

**Author：WILD HORSE**
**数学建模小组成员：**  
队长 贾梓杏 SYSU-物理与天文学院2023级   
队员 夏解金鑫 SYSU-物理与天文学院2023级  
队员 刘张弛 SYSU-数学学院（珠海）2023级  
建模思路在公众号已经分享，此部分为代码展示（不全，但接近全）  
关于data，想来Q1 X和Q1 Y等等含义比较显然的名称就不再解释；此处提供给大家的数据中，我们的结果被我们移除，而是给了大家处理好之后的原始数据  
例如将欧盟的主要国家的CPI加权求和、对年度数据进行插值等等  
如果对Data been Used的部分感兴趣，可以去猜一下文件里面装的是什么--我其实挺建议大家这麽做的，因为拿到手上最初的数据也就那些，甚至还更难看  
希望本文对大家有帮助！感谢大家的支持！也欢迎大家分享给其他小伙伴！  

**启用LaTeX渲染**

In [ ]:
import matplotlib.pyplot as plt
# 启用LaTeX文本渲染
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Computer Modern Roman"],
})

# Question1

### Part One：年度数据插值与PLS回归  
***对大指标进行插值***  
此处的插值逻辑是通用的，改文件地址和列名即可，注意要保留一个year列，是代码索引的关键

In [ ]:
import pandas as pd
import numpy as np
from scipy.interpolate import CubicSpline, interp1d, Akima1DInterpolator
import matplotlib.pyplot as plt

# 读取Excel文件
file_path = "YOUR FILE PATH"
df = pd.read_excel(file_path)



# 将年份转换为标准格式
df.columns = ['year', 'Cat number', 'Dog number', 'Cat total consumption', 'Dog total consumption', 
              'Cat dog food ratio', 'Cat dog food consumption', 'Medical consumption ratio','Other pets amount','GDP per person','Gender ration']

# 检查读取的数据
print("原始数据:")
print(df)

# 进行三次样条插值、线性插值和Akima插值
years = df['year'].values
start_year = years.min()
end_year = years.max()

# 确保years是递增的
if not np.all(np.diff(years) > 0):
    raise ValueError("`years` must be strictly increasing sequence.")

# 计算月份数量
num_months = int((end_year - start_year ) * 12)

# 生成从开始年份到结束年份的月度数据
months = np.linspace(start_year, end_year, num_months)

# 函数进行插值，并确保输出数据范围合理
def interpolate_cubic(column):
    values = df[column].values
    spline = CubicSpline(years, values)
    interpolated_values = spline(months)
    return interpolated_values

def interpolate_linear(column):
    values = df[column].values
    linear_interp = interp1d(years, values, kind='linear')
    interpolated_values = linear_interp(months)
    return interpolated_values

def interpolate_akima(column):
    values = df[column].values
    akima_interp = Akima1DInterpolator(years, values)
    interpolated_values = akima_interp(months)
    return interpolated_values

# 对每一列数据进行插值，并确保插值结果不会出现过大的数值
columns_to_interpolate = ['Cat number', 'Dog number', 'Cat total consumption', 'Dog total consumption', 
                          'Cat dog food ratio', 'Cat dog food consumption', 'Medical consumption ratio','Other pets amount','GDP per person','Gender ration']
interpolated_data_cubic = {col: interpolate_cubic(col) for col in columns_to_interpolate}
interpolated_data_linear = {col: interpolate_linear(col) for col in columns_to_interpolate}
interpolated_data_akima = {col: interpolate_akima(col) for col in columns_to_interpolate}


# 生成包含月度数据的DataFrame（这里选择Akima插值的结果）

dates = pd.date_range(start=f"{int(start_year)}-01-01", periods=num_months, freq='MS')
dom_big_IP = pd.DataFrame({
    'month': dates
})
dom_big_IP = dom_big_IP.assign(**{col: interpolated_data_akima[col] for col in columns_to_interpolate})

# 保存结果到Excel文件
output_path = "D:\\FSS\\MMM\\APMCM\\2024FORcon\\dom_big_IP.xlsx"
dom_big_IP.to_excel(output_path, index=False)

print("月度数据已生成并保存到:", output_path)



*注，上述代码在生成的文件中会多出一列‘month’，使用时删除即可*

**套路化相关性分析**

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 读取Excel文件
file_path ='YOUR FILE PATH'
data = pd.read_excel(file_path)

# 数据清洗和初步检查
print("Data Shape:", data.shape)
print("\nMissing Values:")
print(data.isnull().sum())
print("\nData Types:")
print(data.dtypes)
print("\nDescriptive Statistics:")
print(data.describe())

# 处理缺失值（如果存在）
data.dropna(inplace=True)

# 计算相关性矩阵
correlation_matrix = data.corr()

# 设置绘图风格
sns.set(style='whitegrid')

# 创建画布和轴
plt.figure(figsize=(12, 8))
ax = sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', cbar_kws={'label': r'$\bf{Correlation}$'}, square=True, linewidths=.5, annot_kws={"size": 10})

# 设置标题和标签
plt.title(r'$\bf{Monthly \ Index \ Correlation \ Matrix}$', fontsize=16)
plt.xlabel(r'$\bf{Variables}$', fontsize=14)
plt.ylabel(r'$\bf{Variables}$', fontsize=14)

# 显示图形
plt.tight_layout()
plt.show()




#### 偏最小二乘回归
**数据读取与标准化**  


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_score

# 读取数据
file_path_y = r"D:\FSS\MMM\APMCM\2024FORcon\dom_big_IP.xlsx"
file_path_x = r"D:\FSS\MMM\APMCM\2024FORcon\domestic small index.xlsx"
# YOUR FILE PATH

df_y = pd.read_excel(file_path_y)
df_x = pd.read_excel(file_path_x)

# 提取Y和X的数据
y = df_y.iloc[:, 0].values.reshape(-1, 1)  # 第一个Y指标
x = df_x.iloc[:, :3].values               # 前三个X指标

# 标准化Y和X
scaler_y = StandardScaler()
scaler_x = StandardScaler()

y_scaled = scaler_y.fit_transform(y)
x_scaled = scaler_x.fit_transform(x)


**最佳主成分计算**  
*这里可以注意一下，最后的可视化效果应该是四五个大指标的Cvscores绘制在同一张图上*

In [ ]:
# 使用交叉验证确定最佳主成分数
n_components_range = range(1, 4)
cv_scores = []

for n in n_components_range:
    pls = PLSRegression(n_components=n)
    scores = cross_val_score(pls, x_scaled, y_scaled, cv=5, scoring='neg_mean_squared_error')
    cv_scores.append(scores.mean())

best_n_components = n_components_range[np.argmax(cv_scores)]
print(f"Best number of components: {best_n_components}")


***回归方程计算***

In [ ]:
# 使用最佳主成分数进行PLS回归
pls_final = PLSRegression(n_components=best_n_components)
pls_final.fit(x_scaled, y_scaled.ravel())

# 计算相关系数
coefficients = pls_final.coef_
print("Coefficients for each X variable:")
print(coefficients.T)

### Part Two：小指标LsTm回归  
*数据准备*

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
# 加载数据
file_path = r'D:\FSS\MMM\APMCM\2024FORcon\domestic small index.xlsx'
# YOUR FILE PATH
data = pd.read_excel(file_path)

# 归一化数据
scalers = {}
scaled_data = {}

for column in data.columns:
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_column = scaler.fit_transform(data[[column]])
    scalers[column] = scaler
    scaled_data[column] = scaled_column

# 准备数据集
def create_dataset(dataset, time_step=1):
    X, Y = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        X.append(a)
        Y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(Y)

***模型训练***

In [ ]:
time_step = 12  # 使用前8个月的数据来预测下一个月的数据
Xs, ys = {}, {}

for column in data.columns:
    X, y = create_dataset(scaled_data[column], time_step=time_step)
    Xs[column] = X.reshape(X.shape[0], X.shape[1], 1)
    ys[column] = y

# 构建并训练LSTM模型
models = {}

for column in data.columns:
    model = Sequential()
    model.add(LSTM(units=20, return_sequences=True, input_shape=(Xs[column].shape[1], 1)))
    model.add(LSTM(units=20, return_sequences=False))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(Xs[column], ys[column], epochs=100, batch_size=1, verbose=1)
    models[column] = model

# 评估模型效果
train_predicts, test_predicts = {}, {}

***评估模型效果***

In [ ]:
# 评估模型效果
train_predicts, test_predicts = {}, {}

for column in data.columns:
    train_predict = models[column].predict(Xs[column])
    train_predict = scalers[column].inverse_transform(train_predict)
    train_predicts[column] = train_predict
    
    # 计算训练集上的误差指标
    mse_train = mean_squared_error(ys[column], train_predict)
    rmse_train = np.sqrt(mse_train)
    mae_train = mean_absolute_error(scalers[column].inverse_transform(ys[column].reshape(-1, 1)), train_predict)
    
    print(f"{column} - Train MSE: {mse_train}, RMSE: {rmse_train}, MAE: {mae_train}")
    
    # 创建测试数据集
    x_input = scaled_data[column][-time_step:].reshape(1, -1)
    temp_input_list = list(x_input.flatten())
    
    lst_output = []
    n_steps = time_step
    next_n_days = 36
    
    i = 0
    while(i < next_n_days):
        
        if len(temp_input_list) > n_steps:
            x_input = np.array(temp_input_list[1:])
            x_input = x_input.reshape(1, -1)
            x_input = x_input.reshape((1, n_steps, 1))
            
            yhat = models[column].predict(x_input, verbose=0)
            temp_input_list.extend(yhat[0].tolist())
            temp_input_list = temp_input_list[1:]
            
            lst_output.extend(yhat.tolist())
            i += 1
        else:
            x_input = x_input.reshape((1, n_steps, 1))
            yhat = models[column].predict(x_input, verbose=0)
            temp_input_list.extend(yhat[0].tolist())
            
            lst_output.extend(yhat.tolist())
            i += 1
            
    test_predict = np.array(lst_output).reshape(-1, 1)
    test_predict = scalers[column].inverse_transform(test_predict)
    test_predicts[column] = test_predict.flatten()  # 展平数组 不能用LSTM的格式 去储存

# 将预测结果保存到新的Excel文件中
predictions_df = pd.DataFrame(test_predicts)
# YOUR FILE PATH
predictions_df.to_excel(r'D:\FSS\MMM\APMCM\2024FORcon\dsi_predic.xlsx', index=False)

print("预测结果已保存到 D:\\FSS\\MMM\\APMCM\\2024FORcon\\dsi_predic.xlsx")


***结果可视化***

In [ ]:
plt.figure(figsize=(14, 7))

for column in data.columns:
    plt.plot(range(time_step, len(train_predicts[column]) + time_step), train_predicts[column], label=f'{column} Train Predict')
    plt.plot(range(len(train_predicts[column]) + time_step, len(train_predicts[column]) + time_step + len(test_predicts[column])), 
             test_predicts[column], label=f'{column} Future Predict')
    plt.scatter(range(time_step, len(train_predicts[column]) + time_step), scalers[column].inverse_transform(ys[column].reshape(-1, 1)),
                color='red', s=5, label=f'{column} Actual Values' if column == data.columns[0] else "")

plt.title('Train and Future Prediction Comparison')
plt.xlabel('Time Steps')
plt.ylabel('Values')
plt.legend()
plt.show()

***补充：关于超参数调优***  
超参数调用的思路较多，此处推荐选择不超过五个超参数，使用GA或者网格搜索，如果你有很大算力另当别论

### **Part Three:** 使用X的预测值最终计算Y

In [ ]:
# 读取未来的X数据
file_path_future_x = r"D:\FSS\MMM\APMCM\2024FORcon\dsi_predic.xlsx"
# YOUR FILE PATH
df_future_x = pd.read_excel(file_path_future_x)

# 提取未来的X数据
future_x = df_future_x.iloc[:, :3].values

# 对未来的X数据进行标准化处理
future_x_scaled = scaler_x.transform(future_x)

# 使用训练好的PLS模型进行预测
predicted_y_scaled = pls_final.predict(future_x_scaled)

# 将预测结果反标准化回到原始尺度
predicted_y = scaler_y.inverse_transform(predicted_y_scaled)

# 创建一个DataFrame来存储预测结果
df_predictions = pd.DataFrame(predicted_y, columns=['Predicted_Y'])

# 打印预测结果
print("Predicted Y values (original scale):")
print(df_predictions)

# 可选：将预测结果保存到Excel文件
output_file_path = r"D:\FSS\MMM\APMCM\2024FORcon\predictions.xlsx"
df_predictions.to_excel(output_file_path, index=False)

# Question2  

### Part One：如Q1的PLS  
此处首先补充说明一下欧盟的数据来源。因为在很多数据库里，并不存在“欧盟”的许多数据，而是各个欧洲国家自己的数据  
于是为了不使代表性，我们选取了八个欧洲国家作为欧盟的代表，具体见表格，选取依据是宠物猫狗持有量的占比--FEDIAF报告  
然后根据持有量占比，将各国的指标加权加和。下列代码为取均值。  
并且此处的数据也不建议再动，因为可能改了格式，再去Debug较为麻烦
***欧盟CPI数据处理***

In [ ]:

import pandas as pd

# 读取CSV文件
file_path = r'D:\FSS\MMM\APMCM\2024FORcon\EU CPI.csv'
data = pd.read_csv(file_path)

# 提取出第F列
cpi_data = data['obs_value']

# 倒序排列数据
cpi_data_reversed = cpi_data[::-1]

# 计算每个月份的平均CPI
num_countries = 8
monthly_avg_cpi = []

for i in range(60):
    monthly_cpi_sum = sum(cpi_data_reversed[i::60])
    monthly_avg_cpi.append(monthly_cpi_sum / num_countries)

# 将结果输出到新的CSV文件 'EU CPI av.csv'
output_file_path = r'D:\FSS\MMM\APMCM\2024FORcon\EU CPI av.csv'
pd.DataFrame({'Monthly Avg CPI': monthly_avg_cpi}).to_csv(output_file_path, index=False)





***欧盟UE数据处理***

In [ ]:
import pandas as pd

# 读取CSV文件
file_path = r"D:\FSS\MMM\APMCM\2024FORcon\EU unemployment.csv"
data = pd.read_csv(file_path)

# 提取出第F列（假设列名为'F'）
cpi_data = data['obs_value']

# 倒序排列数据
cpi_data_reversed = cpi_data[::-1]

# 计算每个月份的平均CPI
num_countries = 8
monthly_avg_cpi = []

for i in range(60):
    monthly_cpi_sum = sum(cpi_data_reversed[i::60])
    monthly_avg_cpi.append(monthly_cpi_sum / num_countries)

# 将结果输出到新的CSV文件 'EU CPI av.csv'
output_file_path = r'D:\FSS\MMM\APMCM\2024FORcon\EU unemployment av.csv'
pd.DataFrame({'Monthly Avg CPI': monthly_avg_cpi}).to_csv(output_file_path, index=False)


### Part Two:结合宠物食品成本，先对宠物数量插值再乘以成本
**读取数据**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline

data = {
    'EU': {' Cat': [7900, 8131.1, 8362.2, 7792.8, 7908,7530], 'Dog': [7900.8, 7585.8, 7270.8, 6686.6, 6862,6550]},
    'Russia': {' Cat': [2280, 2275, 2295, 2315, 2326,2250], 'Dog': [1680, 1710, 1755, 1755.8, 1764,1750]},
    'Japan': {' Cat': [977.8, 960, 894.6, 887.3, 906.9,965], 'Dog': [879.7, 850, 710.6, 705.3, 684.4,890]},
    'Brazil': {' Cat': [2710, 2700, 2710, 3360, 3420,2710], 'Dog': [5810, 5800, 5840, 6780, 6840,5810]},
    'America': {' Cat': [9420, 6500, 9420, 7380, 7380,8970], 'Dog': [8970, 8500, 8970, 8970, 8010,9420]}
}
# 提取年份
# years = list(range(2019, 2024))
# months = np.linspace(0, 59, 60)


***三次样条插值***

In [ ]:

# 每个国家每年末的数据点索引
years = np.arange(2018, 2024)

# 新的时间点（每个月）
months = np.linspace(2018, 2023, num=60)

# interpolated_data = {}
for country, animals in data.items():
    interpolated_animals = {}
    for animal, values in animals.items():
        # 确保数据长度一致
        if len(values) != len(years):
            continue
        
        # 创建三次样条插值函数
        cs = CubicSpline(years, values)
        
        # 计算新的月份数据点
        monthly_values = cs(months)
        
        # # 删除第一个数据点（2018年末的数据）
        # monthly_values = monthly_values[1:]
        
        # 生成键
        key = f"{country[0].lower()}_{animal}_pm"
        interpolated_animals[key] = monthly_values
    
    interpolated_data[country.lower()] = interpolated_animals

# 打印结果示例
for country, animals in interpolated_data.items():
    print(f"{country}:")
    for animal, values in animals.items():
        print(f"  {animal}: {values}")


### 储存插值后的数据

In [ ]:
countries = list(data.keys())
pets = set([pet for pet_data in data.values() for pet in pet_data])
print(interpolated_data)


In [ ]:
# 将字典转换为DataFrame
data_frames = {}
for region, data in interpolated_data.items():
    df = pd.DataFrame(data)
    data_frames[region] = df

# 写入Excel文件
with pd.ExcelWriter('interpolated_data.xlsx', engine='openpyxl') as writer:
    for region, df in data_frames.items():
        df.to_excel(writer, sheet_name=region, index=False)

print("数据已成功写入 Excel 文件")


### 绘制图表

***插值后的可视化***

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 设置高级科研配色
sns.set_palette("pastel")

# 读取Excel文件
file_path = r'D:\FSS\MMM\APMCM\2024FORcon\interpolated_data.xlsx'
regions = ['America','Brazil']

# 创建一个新的图形
plt.figure(figsize=(12, 8))

# 定义月份列表
months = pd.date_range(start='2019-01', periods=60, freq='M')


# 绘制每个地区的数据
for i, region in enumerate(regions):
    # 读取每个地区的数据
    df = pd.read_excel(file_path, sheet_name=region)
    
    # 添加日期列
    df['Date'] = months
    
    # 获取颜色
    cat_color = sns.color_palette()[i * 2]
    dog_color = sns.color_palette()[i * 2 + 1]
    
    # 定义线条样式
    cat_linestyle = '-'   # 实线
    dog_linestyle = '-.'  # 虚线
    
    # 绘制猫的数据
    line_cat, = plt.plot(df['Date'], df.iloc[:, 0], label=f'{region} Cats', color=cat_color, linestyle=cat_linestyle, linewidth=2)
    
    # 绘制狗的数据
    line_dog, = plt.plot(df['Date'], df.iloc[:, 1], label=f'{region} Dogs', color=dog_color, linestyle=dog_linestyle, linewidth=2.5)
    
    # 在同一个地区的猫和狗数量曲线之间填充颜色区域
    plt.fill_between(df['Date'], df.iloc[:, 0], df.iloc[:, 1], color=cat_color, alpha=0.2)

# 添加网格
plt.grid(True, which='both', linestyle='-.', linewidth=0.5)


# 设置图形属性
plt.title(r'\textbf{Pet Population Trends }', fontsize=16)
plt.xlabel(r'\textbf{Year}', fontsize=14)
plt.ylabel(r'\textbf{Number of Pets}', fontsize=14)
plt.legend(fontsize=12, title=r'\textbf{Legend}')
plt.tight_layout()

# 显示图形
plt.show()

# Question Three

### Part One ：类似第一问，首先是PLS

### Part Two：还是类似第一问，接着是LSTM的自变量预测  
**为什么这两个Part没有代码呢？因为当时跑代码的时候是直接将第一问的文件地址给改了**

### Part Two：Reg to Export--MLP  
**这里的数据请大家自己跑后进行验证，此处仅为试验。**  
正确的做法是将Global Demand和中国宠物产业生产总值一并作为X使用MLP回归到作为Y的Export上

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# 读取CSV文件
china_exports = pd.read_csv(r'D:\FSS\MMM\APMCM\2024FORcon\China Total Export.csv')
global_demand = pd.read_csv(r'D:\FSS\MMM\APMCM\2024FORcon\Global Total Demand.csv')


# 假设两列数据已经对齐并且有相同的索引
X = global_demand.values.reshape(-1, 1)
y = china_exports.values.reshape(-1, 1)

# 数据标准化
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

# 构建MLP模型
def build_model():
    model = Sequential([
        Dense(64, input_dim=1, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    return model

# 训练和评估模型
model = build_model()
history = model.fit(X_scaled, y_scaled, epochs=100, batch_size=8, validation_split=0.2, verbose=1)

# 使用交叉验证评估模型
cv_scores = cross_val_score(model, X_scaled, y_scaled, cv=5, scoring='neg_mean_squared_error')
cv_rmse_scores = np.sqrt(-cv_scores)

print(f'Cross-validated RMSE scores: {cv_rmse_scores}')
print(f'Mean CV RMSE: {np.mean(cv_rmse_scores)}')





# Question Four

### Event Study

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from datetime import datetime, timedelta

# 读取数据
file_path = 'D:/FSS/MMM/APMCM/2024FORcon/Export_to_A_petfood.xlsx'
# YOUR FILE PATH
data = pd.read_excel(file_path)

# 将Date列转换为日期格式
data['Date'] = pd.to_datetime(data['Date'], format='%Y%m')

# 设置Date为索引，并设置频率
data.set_index('Date', inplace=True)
data = data.asfreq('MS')

# 定义关税变化日期
tariff_change_dates = [pd.Timestamp('2018-10-01'), pd.Timestamp('2019-05-01')]

# 定义时间窗口（五个月）
window_size = 7

# 定义一个函数进行事件研究分析
def event_study_analysis(data, event_date, window_size):
    # 定义估计窗口和事件窗口
    estimation_window_end = event_date - timedelta(days=1)
    estimation_window_start = estimation_window_end - pd.DateOffset(months=window_size)
    event_window_start = event_date
    event_window_end = event_window_start + pd.DateOffset(months=window_size)
    
    estimation_window = data.loc[estimation_window_start:estimation_window_end]
    event_window = data.loc[event_window_start:event_window_end]
    
    # 建立基准模型（使用ARIMA模型）
    model = ARIMA(estimation_window['KG'], order=(3, 1, 3))
    model_fit = model.fit()
    
    # 预测事件窗口内的正常出口量
    forecast = model_fit.get_forecast(steps=len(event_window))
    normal_export_volume = forecast.predicted_mean
    
    # 计算异常出口量
    actual_export_volume = event_window['KG']
    abnormal_export_volume = actual_export_volume - normal_export_volume
    
    # 计算平均异常出口量（AAR）和累积异常出口量（CAR）
    AAR = abnormal_export_volume.mean()
    CAR = abnormal_export_volume.cumsum()
    
    return normal_export_volume, abnormal_export_volume, AAR, CAR, actual_export_volume

# 分析每个关税变化事件
results = {}
for event_date in tariff_change_dates:
    normal_export_volume, abnormal_export_volume, AAR, CAR, actual_export_volume = event_study_analysis(data, event_date, window_size)
    
    results[event_date] = {
        'normal_export_volume': normal_export_volume,
        'abnormal_export_volume': abnormal_export_volume,
        'AAR': AAR,
        'CAR': CAR,
        'actual_export_volume': actual_export_volume
    }
    
    print(f'Event Date: {event_date}')
    print(f'Average Abnormal Returns (AAR): {AAR}')
    print(f'Cumulative Abnormal Returns (CAR): {CAR.iloc[-1]}')
 

# 比较两次关税调整的影响差异
event1_date = tariff_change_dates[0]
event2_date = tariff_change_dates[1]

event1_normal_volume = results[event1_date]['normal_export_volume']
event1_CAR = results[event1_date]['CAR']
event1_actual_volume = results[event1_date]['actual_export_volume']

event2_normal_volume = results[event2_date]['normal_export_volume']
event2_CAR = results[event2_date]['CAR']
event2_actual_volume = results[event2_date]['actual_export_volume']

### Event Study 的可视化

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# 设置图形大小
plt.figure(figsize=(14, 8))

# 定义颜色
color_event2_car = '#FF9999'
color_event1_car = '#66B3FF'
color_normal_volume = '#009E73'
color_actual_volume = '#D55E00'
# 绘制CAR和正常出口量
line1, = plt.plot(event1_CAR.index, event1_CAR, label=f'CAR around {event1_date.date()}', linestyle='--', color=color_event1_car, linewidth=2)
line2, = plt.plot(event2_CAR.index, event2_CAR, label=f'CAR around {event2_date.date()}', linestyle='--', color=color_event2_car, linewidth=2)
line3, = plt.plot(event1_normal_volume.index, event1_normal_volume, label=f'Normal Export Volume around {event1_date.date()}', linestyle='-.', color=color_normal_volume, linewidth=2)
line4, = plt.plot(event2_normal_volume.index, event2_normal_volume, label=f'Normal Export Volume around {event2_date.date()}', linestyle='-.', color=color_normal_volume, linewidth=2)
line5, = plt.plot(event1_actual_volume.index, event1_actual_volume, label=f'Actual Export Volume around {event1_date.date()}', linestyle='-', color=color_actual_volume, linewidth=2)
line6, = plt.plot(event2_actual_volume.index, event2_actual_volume, label=f'Actual Export Volume around {event2_date.date()}', linestyle='-', color=color_actual_volume, linewidth=2)
# 增加阴影效果
plt.fill_between(event1_CAR.index, event1_CAR, color=color_event1_car, alpha=0.3)
plt.fill_between(event2_CAR.index, event2_CAR, color=color_event2_car, alpha=0.3)

# 绘制关税变化事件的垂直线
plt.axvline(event1_date, color='b', linestyle=':', linewidth=2, label='First Tariff Change')
plt.axvline(event2_date, color='r', linestyle=':', linewidth=2, label='Second Tariff Change')

# # 设置X轴标签和Y轴标签
# plt.xlabel('Date', fontsize=14)
# plt.ylabel('Pet Food to US/Kg', fontsize=14)

# # 设置标题
# plt.title('Comparison of Cumulative Abnormal Returns (CAR) with Actual Export Volume', fontsize=16)

# # 调整日期格式
# plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
# plt.xticks(rotation=45)
# 设置X轴标签和Y轴标签，并使用LaTeX语法加粗
plt.xlabel(r'$\bf{Date}$', fontsize=14)
plt.ylabel(r'$\bf{Pet\ Food\ to\ US/Kg}$', fontsize=14)

# 设置标题，并使用LaTeX语法加粗
plt.title(r'$\bf{Comparison\ of\ Cumulative\ Abnormal\ Returns\ (CAR)\ with\ Actual\ Export\ Volume}$', fontsize=16)

# 添加网格
plt.grid(True, linestyle='--', alpha=0.7)

# 绘制第一次加征关税的图例（左上角）
first_tariff_change_legend = plt.legend(handles=[line1, line3, line5],
                                        loc='lower left',
                                        bbox_to_anchor=(0.05 ,0.05),
                                        ncol=1,
                                        title='First Tariff Change',
                                        frameon=True,
                                        shadow=True,
                                        fancybox=True)

# 绘制第二次加征关税的图例（右下角）
second_tariff_change_legend = plt.legend(handles=[line2, line4, line6],
                                         loc='lower right',
                                         bbox_to_anchor=(0.95, 0.05),
                                         ncol=1,
                                         title='Second Tariff Change',
                                         frameon=True,
                                         shadow=True,
                                         fancybox=True)

# 显示图例
plt.gca().add_artist(first_tariff_change_legend)
plt.gca().add_artist(second_tariff_change_legend)

# 自动调整布局
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

# 显示图表
plt.show()

# APPENDICE  
**好看的画图**

In [ ]:
plt.figure(figsize=(10, 9))
col = 'Cat dog food consumption'
plt.plot(years, df[col], 'o', label='Original Data', linewidth=2)
plt.plot(months, interpolated_data_cubic[col], '-', label='Cubic Spline Interpolation', linewidth=2)
plt.plot(months, interpolated_data_linear[col], '--', label='Linear Interpolation', linewidth=2)
plt.plot(months, interpolated_data_akima[col], ':', label='Akima Interpolation', linewidth=2)
plt.grid(True, linestyle='--', alpha=0.7)
plt.title(r'\textbf{Cat dog food consumption}', fontsize=16)
plt.xlabel(r'\textbf{Year}', fontsize=14)
plt.ylabel(r'\textbf{Number/1e8}', fontsize=14)
plt.legend()
plt.tight_layout()
plt.show()